## Momentum-based Trading strategy using Hurst Exponent

In this notebook I've implemented a trading strategy based on Momentum using Hurst Exponent. I've used the Backtesting Toolbox by Auquan.

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Importing the required libraries.

In [48]:
from qq_training_wheels.momentum_trading import MomentumTradingParams
from backtester.trading_system import TradingSystem
from backtester.features.feature import Feature
import numpy as np

## Here's I'm using the 10 days (short term) & 30 days (long term) Momentum along with 90 days Moving Average to make decisions about the trade

## Trading Strategy

I'm going long if the Hurst Exponent is > 0.5 and both long & short term momentum are positive. Similarly I'm going short when the Hurst Exponent is > 0.5 and both long & short term momentum are negative.
When Hurst Exp is < 0.5 I'm using the Mean Reverting Strategy.

In [49]:
class MyTradingFunctions():

    def __init__(self):
        self.count = 0
        # When to start trading
        self.start_date = '2019/12/02'
        # When to end trading
        self.end_date = '2020/12/24'
        self.params = {}

    def getSymbolsToTrade(self):
        '''
            Specifying that I'm trading "only" AAPl stocks.
        '''
        return ['AAPL']

    def getInstrumentFeatureConfigDicts(self):
        '''
            Returns: Array of instrument feature (applied on all instruments) config dictionaries which has the following keys:
                featureId- String representing the type of feature
                featureKey- String representing the key we will use to access the value of this feature
                params- Dictionary with which contains other optional params if needed by the feature
        '''

        ma1Dict = {
            'featureKey': 'ma_90',
            'featureId': 'moving_average',
            'params': {
                'period': 90,
                'featureName': 'adjClose'
            }
        }
        mom30Dict = {
            'featureKey': 'mom_30',
            'featureId': 'momentum',
            'params': {
                'period': 30,
                'featureName': 'adjClose'
            }
        }
        mom10Dict = {
            'featureKey': 'mom_10',
            'featureId': 'momentum',
            'params': {
                'period': 10,
                'featureName': 'adjClose'
            }
        }
        
        return [ma1Dict, mom10Dict, mom30Dict]

    def getPrediction(self, time, updateNum, instrumentManager, predictions):
        '''
            Here we're combining all the above features to create the desired predictions for each stock.
            Note: The output of the prediction function is used by the toolbox to make further trading decisions
                and evaluate our score.
        '''

        self.updateCount() 

        # holder for all the instrument features for all instruments
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
        
        def hurst_f(input_ts, lags_to_test=20): 
            """
                Return the computed Hurst Exponent value
            """
            # hurst < 0.5 - input_ts is Mean Reverting
            # hurst = 0.5 - input_ts is effectively random
            # hurst > 0.5 - input_ts is trending i.e. we can proceed with the Momentum based strategy
            tau = []
            lagvec = []  
            #  Step through the different lags between 2-20 days 
            for lag in range(2, lags_to_test):  
                #  produce price difference with lag  
                pp = np.subtract(input_ts[lag:].values, input_ts[:-lag].values)  
                #  Write the different lags into a vector  
                lagvec.append(lag)  
                #  Calculate the variance of the difference vector  
                tau.append(np.sqrt(np.std(pp)))  
            #  linear fit to double-log graph (gives power)  
            m = np.polyfit(np.log10(lagvec), np.log10(tau), 1)  
            # Finally calculate hurst  
            hurst = m[0]*2
            return hurst  

        # dataframe for a historical instrument feature (ma_90 in this case). The index is the timestamps
        # of upto lookback data points. The columns of this dataframe are the stock symbols/instrumentIds.
        mom10Data = lookbackInstrumentFeatures.getFeatureDf('mom_10')
        mom30Data = lookbackInstrumentFeatures.getFeatureDf('mom_30')
        ma90Data = lookbackInstrumentFeatures.getFeatureDf('ma_90')
        
        # I'm making predictions on the basis of Hurst exponent "only" if enough data is available, otherwise
        # simply get out of our position
        if len(ma90Data.index)>20:
            mom30 = mom30Data.iloc[-1]
            mom10 = mom10Data.iloc[-1]
            ma90 = ma90Data.iloc[-1]
            
            # Calculate the Hurst Exponent
            hurst = ma90Data.apply(hurst_f, axis=0)
            # Go long if Hurst > 0.5 and both long term and short term momentum are positive
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 > 0)] = hurst 
            # Go short if Hurst > 0.5 and both long term and short term momentum are negative
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 <= 0)] = 1-hurst 
            
            # Get out of position if Hurst > 0.5 and long term momentum is positive while short term is negative
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 <= 0)] = 0.5
            # Get out of position if Hurst > 0.5 and long term momentum is negative while short term is positive
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 > 0)] = 0.5
            
            # Mean Revert if Hurst < 0.5
            predictions[(hurst <= 0.5) & mom10<ma90] = hurst 
            predictions[(hurst <= 0.5) & mom10>ma90] = 1-hurst        
        else:
            # If no sufficient data then don't take any positions
            predictions.values[:] = 0.5
        return predictions

    def updateCount(self):
        self.count = self.count + 1

### Initializing the Trading System

In [50]:
tf = MyTradingFunctions()
tsParams = MomentumTradingParams(tf)
tradingSystem = TradingSystem(tsParams)

Processing data for stock: AAPL
20% done...
40% done...
60% done...
80% done...
Logging all the available market metrics in tensorboard
Logging all the available instrument metrics in tensorboard


## Let's start Trading!!

In [51]:
results = tradingSystem.startTrading()

2019-12-02 00:00:00
2019-12-03 00:00:00
AAPL
pnl: 0.00
2019-12-04 00:00:00
AAPL
pnl: 0.00
2019-12-05 00:00:00
AAPL
pnl: 0.00
2019-12-06 00:00:00
AAPL
pnl: 0.00
2019-12-09 00:00:00
AAPL
pnl: 0.00
2019-12-10 00:00:00
AAPL
pnl: 0.00
2019-12-11 00:00:00
AAPL
pnl: 0.00
2019-12-12 00:00:00
AAPL
pnl: 0.00
2019-12-13 00:00:00
AAPL
pnl: 0.00
2019-12-16 00:00:00
AAPL
pnl: 0.00
2019-12-17 00:00:00
AAPL
pnl: 0.00
2019-12-18 00:00:00
AAPL
pnl: 0.00
2019-12-19 00:00:00
AAPL
pnl: 0.00
2019-12-20 00:00:00
AAPL
pnl: 0.00
2019-12-23 00:00:00
AAPL
pnl: 0.00
2019-12-24 00:00:00
AAPL
pnl: 0.00
2019-12-25 00:00:00
AAPL
pnl: 0.00
2019-12-26 00:00:00
AAPL
pnl: 0.00
2019-12-27 00:00:00
AAPL
pnl: 0.00
2019-12-30 00:00:00
AAPL
pnl: 0.00
2019-12-31 00:00:00
AAPL
pnl: 0.00
2020-01-01 00:00:00
AAPL
pnl: 0.00
Position changed to: -1.00
2020-01-02 00:00:00
AAPL
pnl: 0.00
2020-01-03 00:00:00
AAPL
pnl: -1.66
2020-01-06 00:00:00
AAPL
pnl: -0.94
2020-01-07 00:00:00
AAPL
pnl: -1.52
2020-01-08 00:00:00
AAPL
pnl: -1.17
2020

In [52]:
results

{'instrument_names': ['AAPL'],
 'instrument_stats': [{'pnl': {'AAPL': 0.05950761499999997},
   'score': {'AAPL': -0.07723594387228924}}],
 'pnl': 0.05950761499999997,
 'trading_days': 278,
 'annual_return': 0.05379520628556933,
 'annual_vol': 0.03765671312821343,
 'sharpe_ratio': 1.4285688212459655,
 'score': -0.07723594387228924,
 'total_profit': 242.06572700000004,
 'variance': 5.627095411192704,
 'maxDrawdown': 27.292907000000014,
 'maxPortfolioValue': 1062.496506,
 'capital': 875.5879933500004,
 'capitalUsage': 127.25145309999971,
 'total_loss': 182.558112,
 'count_loss': 106,
 'portfolio_value': 1059.507615,
 'count_profit': 128}

## Interpret the Results

We see that the PnL is just shy of 6 %. And Sharp Ratio is 1.42.